In [348]:
from os import path
from collections import defaultdict
import parse

stdout = list()

with open(path.join(globals()['_dh'][0], "input.txt")) as f:
    stdout = [x.strip() for x in f.readlines()]

stdout_test = list()
with open(path.join(globals()['_dh'][0], "test.txt")) as f:
   stdout_test = [x.strip() for x in f.readlines()]

In [35]:
LIST = 'ls'
CHANGE_DIRECTORY = 'cd'

command_template = parse.compile('$ {command}')

In [10]:
def is_command(command_string):
    return '$' == command_string[0]

In [56]:
def parse_command(line):
    raw_command = command_template.parse(line)['command'].split()

    command, args = tuple(raw_command if len(raw_command) == 2 else [raw_command[0], None])

    return command, args

In [155]:
def parse_content(line):
    size, filename = line.split()
    
    return size, filename

In [426]:
def parse_stdout(stdout):
    directory_stack = list()
    
    directory_map = defaultdict(set)
    
    for line in stdout:
        depth_index = len(directory_stack)
        if is_command(line):
            command, args = parse_command(line)

            if args == '..':
                directory_stack.pop()
            elif not args: continue
            else:
                directory_stack.append(args)
        else:
            size, filename = parse_content(line)
            
            current_dir = directory_stack[-1]
            parent_dir = directory_stack[-2] if len(directory_stack) > 1 else None
            directory_map[((parent_dir, depth_index - 1), depth_index, current_dir)].add((size == 'dir', (current_dir, depth_index), size, (depth_index, filename)))

    return directory_map

In [343]:
def get_directory_size(directory, directory_map):
    result = 0
    for file in directory_map[directory]:
        is_directory, origin, size, filename = file

        if is_directory:
            result += get_directory_size((origin, origin[1] + 1, filename[1]), directory_map)
        else:
            result += int(size)
    
    return result

In [336]:
def print_structure(directory, directory_map):
    for file in directory_map[directory]:
        is_directory, origin, size, filename = file

        if is_directory:
            print(f"{'  ' * filename[0]}- {filename[1]} (dir)")
            print_structure((origin, origin[1] + 1, filename[1]), directory_map)
        else:
            print(f"{'  ' * filename[0]}- {filename[1]} (file, size={size})")

In [423]:
def part1(stdout):
    directory_map = parse_stdout(stdout)

    directory_sizes = [get_directory_size(directory, directory_map) for directory in list(directory_map.keys())]

    return sum([size for size in directory_sizes if size <= 100000])

In [421]:
def part2(stdout):
    directory_map = parse_stdout(stdout)

    directory_sizes = [get_directory_size(directory, directory_map) for directory in list(directory_map.keys())]
    directory_sizes.sort()
    directory_sizes = directory_sizes[::-1]
    
    free_space = 70000000 - directory_sizes[0]
    space_needed = 30000000 - free_space

    return min([x for x in directory_sizes[1:] if x >= space_needed])


In [422]:
print(f"Solution to part 1 is {part1(stdout)}")
print(f"Solution to part 2 is {part2(stdout)}")

Solution to part 1 is 2031851
Solution to part 2 is 2568781
